### <h1 align="center" id="title">IGM notebook tutorial (aletsch-simple) </h1>

This tutorial aims to guide you modelling the great Aletsch Glacier, Switzerland, in a very simple set-up with the Instructed Glacier Model (IGM).

Let us firt download the IGM code, ice flow emulator, and input data

In [4]:
# get the core source of igm (in igm.py)
!wget -O igm.py https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py

# get input data
!wget -O geology.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-simple/geology.nc
!wget -O mb_simple_param.txt https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-simple/mb_simple_param.txt

# get the ice flow emulator
!apt install subversion
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f12_cfsflow_GJ_21_a

# force version 2.4 of Tensorflow
%tensorflow_version 2.4

--2021-12-22 07:30:43--  https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47154 (46K) [text/plain]
Saving to: ‘igm.py’

igm.py              100%[===================>]  46.05K  --.-KB/s    in 0.01s   

2021-12-22 07:30:43 (4.57 MB/s) - ‘igm.py’ saved [47154/47154]

--2021-12-22 07:30:43--  https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-simple/geology.nc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373742 (365K) [application/octet-stream]
Saving to:

In the first set-up, we take top ice surface topography and (zero) ice thickness defined in geology.nc, and force IGM with mass balance parameters defined in mb_simple_param.txt.

First, we import necessary libraries, check the version of tensorflow, and inport the class igm (defined in igm.py), and create an igm object.

In [5]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print('TF version is ',tf.__version__)
sys.argv = ['']  # this is absolutly necessary in Jupyter notebook
from igm import *
igm=igm() # create a class object igm

TF version is  2.7.0


Altough, you don't have to, it is highly recommended to run under GPU (click above 'Runtime' -> 'Change Runtime type' -> Activate GPU). Then the fowllowing comand check that a GPU is indeed available, and give the name of the GPU. (With the free veryion of colab these GPU are often old and not that efficient -- like the K80 --, IGM would typically run faster on new material)

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6554490505871063433
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12693889508590080714
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

Next, we overide the default configuration parameters, initialize the class, and check at the parameters

In [6]:
igm.config.tstart            = 1000  # we start at time t=1000
igm.config.tend              = 1250  # we end at time   t=1250
igm.config.tsave             = 5     # we save result each 5 years
igm.config.init_strflowctrl  = 90    # we fix the ice flow strenght control to 90 (i.e. A=78, c=12)
igm.config.iceflow_model_lib_path    = 'f12_cfsflow_GJ_21_a' # we provide the path for th ice flow emultor
igm.config.type_mass_balance = 'simple' # we use the 'simple' ELA-parametrized SMB model
igm.config.mb_simple_file    = 'mb_simple_param.txt' # this is the file where are defined the SMB params.
igm.config.usegpu            = True # we do not use the GPU for the first run
igm.config.plot_live         = False # we do not plot the result life, but vizualize a posteriori
igm.config.plot_result       = False # we do not plot the result life, but vizualize a posteriori

igm.initialize()

+++++++++++++++++++ START IGM ++++++++++++++++++++++++++++++++++++++++++
PARAMETERS ARE ...... 
                   working_dir : 
                  geology_file : geology.nc
                        tstart : 1000
                          tend : 1250
                restartingfile : 
                     verbosity : 0
                         tsave : 5
                   plot_result : False
                     plot_live : False
                        usegpu : True
                          stop : False
              init_strflowctrl : 90
                      optimize : False
                   update_topg : False
              clim_update_freq : 1
                  type_climate : 
        iceflow_model_lib_path : f12_cfsflow_GJ_21_a
          multiple_window_size : 0
              force_max_velbar : 0
                  vars_to_save : ['topg', 'usurf', 'thk', 'smb', 'velbar_mag', 'velsurf_mag']
                       varplot : velbar_mag
                   varplot_max : 500
          

Now we run the IGM time evolution workflow with igm.run(), which is strictly equivalent to the commented list of operations and permits to vizualize all steps within the time loop.


In [8]:
igm.run() # is equivalent to the following comand list

# with tf.device(igm.device_name):
#    igm.load_ncdf_data(igm.config.geology_file)
#    igm.initialize_fields()               
#    igm.initialize_iceflow()
#    igm.update_iceflow()                    
#    igm.update_ncdf_ex()
#    igm.print_info()
#    while igm.t < igm.config.tend:                       
#        igm.update_smb()
#        igm.update_iceflow()
#        igm.update_t_dt() 
#        igm.update_thk()       
#        igm.update_ncdf_ex()
#        igm.update_plot()
#        igm.print_info()

+++++++++++++++++++ START IGM ++++++++++++++++++++++++++++++++++++++++++
PARAMETERS ARE ...... 
                   working_dir : 
                  geology_file : geology.nc
                        tstart : 1000
                          tend : 1250
                restartingfile : 
                     verbosity : 0
                         tsave : 5
                   plot_result : False
                     plot_live : False
                        usegpu : True
                          stop : False
              init_strflowctrl : 90
                      optimize : False
                   update_topg : False
              clim_update_freq : 1
                  type_climate : 
        iceflow_model_lib_path : f12_cfsflow_GJ_21_a
          multiple_window_size : 0
              force_max_velbar : 0
                  vars_to_save : ['topg', 'usurf', 'thk', 'smb', 'velbar_mag', 'velsurf_mag']
                       varplot : velbar_mag
                   varplot_max : 500
          

We have now modelled 250 years of evolution of the Great Aletsch Glacier in less than a minute! (this would take < 15 sec on modern GPU). IGM monitors key variables (ice volume, time step) during computation you can monitor, and gives a statatistic of the computational cost of each mdel components at the end. The results were stored in ex.nc. Now we can vizualize the results (ice thickness, ice speed, or more) in form of an animation.



In [23]:
from IPython.display import HTML, display
import xarray as xr 
from matplotlib import pyplot as plt, animation

# open the thickness variable from the netcdf file (choose ice thcikness or ice speed)
#var  = xr.open_dataset('ex.nc',engine='netcdf4').thk            ; maxvar = 800
var  = xr.open_dataset('ex.nc',engine='netcdf4').velbar_mag    ; maxvar = 250
 
fig, ax = plt.subplots(figsize=(8,10))

# Plot the initial frame. 
cax = var[0,:,:].plot(add_colorbar=True,cmap=plt.cm.get_cmap('jet', 10),vmin=0, vmax=maxvar)
ax.axis("off") ; ax.axis("equal")

# dont' show the original frame
plt.close()

def animate(i):
    cax.set_array(var[i,:,:].values.flatten())
    ax.set_title("Time = " + str(var.coords['time'].values[i])[:13])

ani = animation.FuncAnimation( fig, animate, frames=var.shape[0], interval=100 ) # interval in ms between frames

HTML(ani.to_html5_video())

# optionally the animation can be saved in avi
# ani.save('animation.mp4')

igm.config.type_mass_balance = 'sinus'
Now, we may custumize your our mass balance function with an oscillating ELA. For that, you can define a function of the class igm, and make sure to activate the corresponding keywork in igm.config.type_mass_balance

In [ ]:
!rm ex.nc

In [ ]:
from igm import igm
import math

class igm(igm):

    def update_smb_sinus(self):
        """
            mass balance 'sinus'
        """

        ela     = 2800 + 500*math.sin((self.t/50)*math.pi)
        gradabl = 0.009
        gradacc = 0.005
        maxacc  = 2.0

        smb = self.usurf - ela
        smb *= tf.where(tf.less(smb, 0), gradabl, gradacc)
        smb = tf.clip_by_value(smb, -100, maxacc)
        smb = tf.where(self.icemask>0.5, smb, -10 )

        self.smb.assign( smb )

igm = igm()

igm.config.tstart                 = 0
igm.config.tend                   = 500
igm.config.tsave                  = 2
igm.config.cfl                    = 0.5
igm.config.init_strflowctrl       = 90
igm.config.iceflow_model_lib_path = 'f12_cfsflow_GJ_21_a'
igm.config.type_mass_balance      = 'sinus'
igm.config.usegpu                 = True

igm.run()